In [ ]:
# LangChain 实战：家装销售聊天机器人

In [1]:
with open("house_decoration_sales_data.txt",encoding="utf-8") as f:
    house_decoration_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([house_decoration_sales])

In [5]:
docs[0]

Document(page_content='[客户问题]: 这个价格是最终的吗？\n[销售回答]: 这个价格已经包含了所有的材料和安装费用，我们保证提供最具竞争力的价格和最优质的服务。', metadata={})

In [6]:
len(docs)

70

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [8]:
query = "环保达标吗"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]: 你们的产品环保吗？
[销售回答]: 我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。

[客户问题]: 这款产品的生产过程环保吗？
[销售回答]: 我们致力于环保生产，所有的生产过程都符合最严格的环保标准。

[客户问题]: 你们的产品对环境有影响吗？
[销售回答]: 我们非常关心环境问题，我们的产品都是在符合环保标准的条件下生产的。我们始终努力减少对环境的影响。

[客户问题]: 这款产品能持久保持新的状态吗？
[销售回答]: 是的，只要按照我们提供的保养指南进行定期清洁和保养，产品就能长期保持新的状态。



In [11]:
db.save_local("house_decorations_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x000001B0DD844610>, search_type='similarity', search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 你们的产品环保吗？
[销售回答]: 我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。

[客户问题]: 这款产品的生产过程环保吗？
[销售回答]: 我们致力于环保生产，所有的生产过程都符合最严格的环保标准。

[客户问题]: 你们的产品对环境有影响吗？
[销售回答]: 我们非常关心环境问题，我们的产品都是在符合环保标准的条件下生产的。我们始终努力减少对环境的影响。



In [15]:
docs = topK_retriever.get_relevant_documents("你们有没有100万的橱柜啊？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 我可以在你们这里买到所有我需要的家具吗？
[销售回答]: 我们提供全方位的家装产品，无论您需要什么样的家具，我们都能为您提供。

[客户问题]: 你们的产品符合国际标准吗？
[销售回答]: 是的，我们的产品都符合国际家具标准，我们致力于为客户提供最高质量的产品。

[客户问题]: 你们的家具耐用吗？
[销售回答]: 我们的家具都是由优质材料制成，经过精心设计和制造，耐用度非常高。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [21]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

In [22]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 你们的产品环保吗？
[销售回答]: 我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。

[客户问题]: 这款产品的生产过程环保吗？
[销售回答]: 我们致力于环保生产，所有的生产过程都符合最严格的环保标准。

[客户问题]: 你们的产品对环境有影响吗？
[销售回答]: 我们非常关心环境问题，我们的产品都是在符合环保标准的条件下生产的。我们始终努力减少对环境的影响。

[客户问题]: 这款产品能持久保持新的状态吗？
[销售回答]: 是的，只要按照我们提供的保养指南进行定期清洁和保养，产品就能长期保持新的状态。



### 提取向量数据库中的`销售回答`

In [23]:
docs = retriever.get_relevant_documents(query)

In [28]:
docs[0].page_content

'[客户问题]: 你们的产品环保吗？\n[销售回答]: 我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。'

In [30]:
docs[0].page_content.split("[销售回答]: ")

['[客户问题]: 你们的产品环保吗？\n', '我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。']

In [31]:
ans = docs[0].page_content.split("[销售回答]: ")[-1]

In [32]:
ans

'我们所有的产品都符合最严格的环保标准，我们致力于提供环保且健康的产品。'

#### 尝试各种问题

In [33]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]: ")[-1] for doc in docs]

    return ans_list

In [36]:
query = "我想要个白色的壁布"

print(sales(query))

[]


In [37]:
print(sales(query, 0.75))

[]


In [38]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []

score:0.5 ans: ['这个价格已经包含了所有的材料和安装费用，我们保证提供最具竞争力的价格和最优质的服务。', '我们的价格包括了产品和安装服务，您不需要额外支付安装费用。', '是的，如果您计划购买大量的产品，我们可以为您提供一些优惠。具体的折扣详情，我们可以进一步讨论。', '当然，您可以随时来我们的实体店查看产品，我们的销售团队会为您提供详细的产品介绍。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [39]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [40]:
qa_chain({"query": query})

{'query': '价格200万以内', 'result': '请问您是在询问什么类型的物品或服务的价格在200万以内呢？'}

In [41]:
qa_chain({"query": "有智能窗帘吗？"})

{'query': '有智能窗帘吗？',
 'result': '是的，有智能窗帘。智能窗帘是一种可以通过智能设备控制的窗帘，可以实现自动开关、定时控制、遥控操作等功能。'}

In [43]:
print(sales("有智能窗帘吗"))

[]


## 加载 FAISS 向量数据库已有结果

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("house_decorations_sale", OpenAIEmbeddings())

In [45]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [46]:
qa_chain({"query": "我想买个智能窗帘，你们有么"})

{'query': '我想买个智能窗帘，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供实际产品。建议您在当地的家居商店或在线购物平台上搜索智能窗帘，以找到合适的产品。'}

In [47]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [48]:
qa_chain({"query": "我想买个智能窗帘，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买个智能窗帘，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供实际产品。建议您在当地的家居商店或在线购物平台上搜索智能窗帘，以找到合适的产品。'}

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True